This notebook was used to generate the inputs and expected outputs for fir_test

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import numpy as np
import plotly.graph_objects as go
from rtmha.filter import FirFilter
from rtmha.elevenband import elevenband_taps_min

In [ ]:
def plot_res(x,y, res):
    ms = np.linspace(0,10,len(x))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=ms, y=y, name='input'))
    fig.add_trace(go.Scatter(x=ms, y=res, name='output'))
    fig.update_layout(xaxis_title='milliseconds',
                      yaxis_title='Amplitude',
                      template='plotly_dark')
    fig.show()

In [ ]:
# this is the band 0 filter, so the rate is downsampled to 1/16
sample_rate = 32000 
down_rate=sample_rate / 16 
nyq_rate=down_rate / 2 

In [ ]:
def generate_sine_waves(freq_list, duration=1, sample_rate=32000):
    """Generates a signal with multiple sine waves

    Args:
        freq_list: List of frequencies
        duration: signal length in seconds (default 1)
        sample_rate: sample rate in Hz (default 32000)

    Returns:
        (t, y): t is time. y is value in range [-1,1]
    """
    x = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    y = np.zeros(len(x))
    for freq in freq_list:
        frequencies = x * freq
        y += np.sin((2 * np.pi) * frequencies)
    y /= len(freq_list)   # normalize
    return x, y

In [ ]:
x, y = generate_sine_waves([200, 4000], duration=32*10/down_rate, sample_rate=down_rate)

In [ ]:
f = FirFilter(elevenband_taps_min[0], len(y))
res = f.filter(y)
plot_res(x,y,res)

In [ ]:
from scipy.signal import lfilter
out = lfilter(elevenband_taps_min[0], 1.0, y)
plot_res(x, y, out)

In [ ]:
np.allclose(out, res)

In [ ]:
f = FirFilter(elevenband_taps_min[10], len(y))
res = f.filter(y)
plot_res(x,y,res)

In [ ]:
inp = np.zeros(128).astype('float32')
inp[1]=1
f = FirFilter(elevenband_taps_min[0], 128)
res = f.filter(inp)
plot_res(inp, inp,res)

In [ ]:
%%timeit 
res = f.filter(inp)

In [ ]:
%%timeit 
out = lfilter(elevenband_taps_min[0], 1.0, inp)

In [ ]:
out = lfilter(elevenband_taps_min[0], 1.0, inp)
plot_res(inp, inp, out)

In [ ]:
f = FirFilter(elevenband_taps_min[10], len(inp))
res = f.filter(inp)
plot_res(inp, inp,res)

In [ ]:
down_rate = 32000
x, y = generate_sine_waves([8000], duration=32*4/down_rate, sample_rate=down_rate)

In [ ]:
f = FirFilter(elevenband_taps_min[10], len(y))
res = f.filter(y)
plot_res(y, y,res)